## scrap

In [101]:
!pip install datasets


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/7c/55/b3432f43d6d7fee999bb23a547820d74c48ec540f5f7842e41aa5d8d5f3a/datasets-2.14.6-py3-none-any.whl.metadata
  Obtaining dependency information for xxhash from https://files.pythonhosted.org/packages/33/db/e07aa80e39a7ee82e9ca6f5a0fa9bf0b4465934272116a1b578eaee7f4b3/xxhash-3.4.1-cp39-cp39-win_amd64.whl.metadata
  Obtaining dependency information for multiprocess from https://files.pythonhosted.org/packages/c6/c9/820b5ab056f4ada76fbe05bd481a948f287957d6cbfd59e2dd2618b408c1/multiprocess-0.70.15-py39-none-any.whl.metadata
  Obtaining dependency information for dill<0.3.8,>=0.3.0 from https://files.pythonhosted.org/packages/f5/3a/74a29b11cf2cdfcd6ba89c0cecd70b37cd1ba7b77978ce611eb7a146a832/dill-0.3.7-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/493.7 kB ? eta -:--:--
   -------------- ------------------------- 174.1/493.7 kB 5.3 MB/s eta 0:00:01
   --------------------

In [1]:
!pip install apiclient



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
from googleapiclient.discovery import build
from langdetect import detect
from googleapiclient.errors import HttpError
import pandas as pd
import re
from datetime import datetime
from urllib.parse import urlparse, parse_qs


DEVELOPER_KEY = "Set your YouTube Data API key here"

YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

# Initialize the YouTube Data API client
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)

def extract_video_id(url):
    query = urlparse(url)
    if query.hostname == 'youtu.be':
        return query.path[1:]
    if query.hostname in ('www.youtube.com', 'youtube.com'):
        if query.path == '/watch':
            p = parse_qs(query.query)
            return p['v'][0]
        if query.path[:7] == '/embed/':
            return query.path.split('/')[2]
        if query.path[:3] == '/v/':
            return query.path.split('/')[2]
    return None

def get_comments(video_id, max_comments=10):
    comments_data = []
    try:
        # Get comments for the specified video
        response = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=max_comments
        ).execute()

        # Extract and store the comments, their upload dates, and usernames
        for comment in response.get("items", []):
            comment_text = comment["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
            comment_date = comment["snippet"]["topLevelComment"]["snippet"]["publishedAt"]
            username = comment["snippet"]["topLevelComment"]["snippet"]["authorDisplayName"]

            # Check if the comment is in Arabic
            clean_comment = re.sub(r'[^\w\s]', '', comment_text)
            if len(clean_comment) >= 3:
                if detect(clean_comment) == "ar":
                    formatted_date = datetime.strptime(comment_date, "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%d")
                    comments_data.append({"date": formatted_date, "text": comment_text, "username": username})

    except HttpError as e:
        print("An HTTP error occurred:", e)

    return comments_data

def get_comments_from_url(url, max_comments=100):
    video_id = extract_video_id(url)
    if video_id:
        return get_comments(video_id, max_comments)
    else:
        print("Invalid YouTube URL")
        return []

# Specify the YouTube video URL here
youtube_url = "https://youtu.be/IrkFeijDREk?si=8bNXdys7TmQFdjrO"

# Call the function to get comments from the video URL
video_comments_data = get_comments_from_url(youtube_url, max_comments=100)

# Create a DataFrame from the collected comments data
video_comments_df = pd.DataFrame(video_comments_data, columns=["date", "text", "username"])

# Add a "source" column and save to Excel
source_value = "YouTube"
video_comments_df["source"] = source_value
video_comments_df.to_excel('video_comments.xlsx', index=False)


In [45]:
video_comments_df

,date,text,username,source
0,2023-10-24,شكون لي رجع لايفاة قدام 😢😊,Simo Rajawi,YouTube
1,2023-10-08,الله إسهل على الجميع❤❤❤❤❤❤,Maroc Kenitra,YouTube
2,2023-09-01,الاخوان اش سميت الكونط تاعوا,الساموراي_samurai,YouTube
3,2023-07-25,خوتي ممكن ديرو لي ابوني,Mustahdaf. gamer,YouTube
4,2023-07-21,إلياس تزعط فهاد خيي😂😂😂😂😂,Rime Goumiry,YouTube
5,2023-07-20,كين😂,Younes Ghandi,YouTube


In [32]:
import pandas as pd

# Concatenate df1 and df2 along the rows (axis=0)
df = pd.concat([df,video_comments_df])

df = df.reset_index(drop=True)




In [41]:
df

,date,text,username,source
0,2023-07-15,ضحكتو كبر منو هه طوفو,mc jabran,YouTube
1,2023-07-14,عادي ينزل عليك بشي سطرايك و حسسسسسان,Kiki,YouTube
2,2023-07-13,متسبش الدين,ali lahmama,YouTube
3,2023-07-13,واحساااان 😂,Princesse,YouTube
4,2023-07-12,ناري العزوا لا نفارد 😂😂😂,EL HCM Tv,YouTube
5,2023-07-11,الياس علام والله 🤣🤣 ما تقدرش يقول شحاجة وما تضحكش,BADI FATH ALLAH,YouTube
6,2023-10-18,مرر رسائل لارضاء الحاضرين خاصة ناهبي الشعب من ...,دخل تشوف,YouTube
7,2023-09-22,هذا ما هو فنان و لا حثى حلايقي و ليس له موهبة ...,Aboulahab Koraychi,YouTube
8,2023-09-17,سكيتش تحت التافه,hassoun didaw,YouTube
9,2023-09-03,واش هذا فكاهي<br>الله ينعل اللي ما يحشم,Jamal Barquacha,YouTube


In [93]:
import pandas as pd

df.to_excel('tanamor.xlsx', index=False)

In [95]:
data =pd.read_excel(r"C:\Users\HP\OneDrive\Bureau\hackathon isic\tanamor.xlsx")

In [97]:
!pip install pyarabic

     ---------------------------------------- 0.0/126.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/126.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/126.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/126.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/126.4 kB ? eta -:--:--
     --------- ------------------------------ 30.7/126.4 kB ? eta -:--:--
     --------- ------------------------------ 30.7/126.4 kB ? eta -:--:--
     --------- ------------------------------ 30.7/126.4 kB ? eta -:--:--
     --------- ------------------------------ 30.7/126.4 kB ? eta -:--:--
     --------- ------------------------------ 30.7/126.4 kB ? eta -:--:--
     --------- ------------------------------ 30.7/126.4 kB ? eta -:--:--
     ----------------- ------------------- 61.4/126.4 kB 192.5 kB/s eta 0:00:01
     ----------------- ------------------- 61.4/126.4 kB 192.5 kB/s eta 0:00:01
     ----------------- --------


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [98]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pyarabic.araby import strip_tashkeel, strip_tatweel
# Download the required nltk resources (only required once)
nltk.download('punkt')
nltk.download('stopwords')
def preprocess_arabic_text(text):
    text = strip_tashkeel(text)
    text = strip_tatweel(text)

    additional_symbols = r'[،؟]'  

    pattern = r'[' + re.escape(additional_symbols) + ']'
    text = re.sub(pattern, '', text)
    
    # Remove non-Arabic characters and numbers
    text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
    
    words = word_tokenize(text)
    
    stop_words = set(stopwords.words('arabic'))
    words = [word for word in words if word not in stop_words]
    
    preprocessed_text = ' '.join(words)
    
    return preprocessed_text



    
    
data['Text_pro'] = data['text'].apply(preprocess_arabic_text)
    

   

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [99]:
data

,date,text,username,source,Topic,label,Text_pro
0,2023-10-06,واوو,Driss Elbasry,YouTube,NV,0,واوو
1,2023-10-01,الغالب اللله,Idrissia Echchahid,YouTube,NV,0,الغالب اللله
2,2023-09-20,الوسخ,Najat Rifia,YouTube,V,1,الوسخ
3,2023-09-17,ختي خليك غير فالتمتيل اش داك لشي غنا انا بعدا ...,Kati Zora,YouTube,V,1,ختي خليك فالتمتيل اش داك لشي غنا انا حشمت نغني...
4,2023-09-05,لا علاقة تحربيق,Fati Fati,YouTube,V,1,علاقة تحربيق
...,...,...,...,...,...,...,...
65,2023-07-07,كون شي مغربي كانت اكبر معاه هادشي !! ههههه مع ...,Mouhssine Hd's,YouTube,V,1,كون شي مغربي كانت اكبر معاه هادشي ههههه البران...
66,2023-07-01,امتى العرس❤,BL Ikram,YouTube,NV,0,امتى العرس
67,2023-06-28,كلشي ما فهمت يحجون. كردي انتم فهمتو😂❤,حسين عوده,YouTube,NV,0,كلشي فهمت يحجون كردي انتم فهمتو
68,2023-06-08,شحال غزالين مع بعضهم ❤❤,Mina Amina,YouTube,NV,0,شحال غزالين بعضهم


In [100]:
import pandas as pd

data.to_excel('official_dataset.xlsx', index=False)